In [26]:
import numpy as np
import pandas as pd
import os

In [42]:
#crate paths for csv files
CURR_DIR = os.getcwd()
print(CURR_DIR)
zlecenia_path = os.path.join(CURR_DIR, 'Zlecenia.csv')
godziny_path = os.path.join(CURR_DIR, 'Godziny.csv')

/Users/w.szczawinski/Projects/DS_final_project


In [41]:
#Zaciagniecie baz danych
obciazenie_df = pd.read_csv(zlecenia_path, sep=';')
godziny_df = pd.read_csv(godziny_path, sep=';')

In [30]:
#Ekstrakcja danych i organizacja danych - pozostawienie tylko kolumn wymaganych, pozostawienie tylko projektów "w toku" - nie działa
obciazenie_df1 = obciazenie_df.drop(columns=['work_preparation_Plan','work_preparation_done'], axis=1)
obciazenie_df2 = obciazenie_df1.loc[obciazenie_df1['Active'].isin(['Active'])]

In [31]:
#Left join po No z godzinami
obciazenie_df2 = pd.merge(obciazenie_df2, godziny_df, how='left', on='No')

#TEST!!!!! wypelnienie NaN zerami
#obciazenie_df2['Wszystkie.technol'] = obciazenie_df2['Wszystkie.technol'].fillna(0)
#obciazenie_df2['Wszystkie.wykonane'] = obciazenie_df2['Wszystkie.wykonane'].fillna(0)

In [32]:
#Zdefiniowanie tygodnia obecnego
#przyjmujemy obecny jako 202112(tydzień zaciągnięcia danych),
#docelowo może zostać zamieniony funkcją obliczającą jak poniżej
#WPISAĆ FUNKCJĘ!!!!!!!!!

current_week = 202112



In [33]:
#Zmiana terminu rozpoczęcia oraz zakończenia na obecny jeżeli miało mieć ono miejsce w przeszłości
# !!!NIE DZIAŁA!!!!

#if  obciazenie_df2['Finnish_week'] < current_week:
#    obciazenie_df2['Finnish_week'] == current_week
#else:
#    obciazenie_df2['Finnish_week'] == obciazenie_df2['Finnish_week']
    
#if  obciazenie_df2['Pre_fabrication_plan'] < current_week:
#    obciazenie_df2['Pre_fabrication_plan'] = current_week
#else:
#    obciazenie_df2['Pre_fabrication_plan'] = obciazenie_df2['Finnish_week']

In [34]:
#Zastapienie NaN w godzinach wartościami przewidzianymi z prekalkulacji

#A - dziób(wędka)
#Labor rate	130 rbg/1000kg
#Assembling	0,4
#Welding	0,33
#Grindining	0,07
#Else		0,2

#B - belka(środek)
#Labor rate	100 rbg/1000kg
#Assembling	0,3
#Welding	0,4
#Grindining	0,2
#Else		0,1

#C - końcóka(środek)
#Labor rate	120 rbg/1000kg
#Assembling	0,25
#Welding	0,4
#Grindining	0,2
#Else		0,15

# IF Wszystkie technol = NaN -> Weight for 1 * Quantity * Typ (IF A - 0,13 elif B - 0,1 elif C - 0,12)

In [35]:
#Obliczenie kolumn - pozostałego czasu na produkcję oraz pozostałych godzin jeżeli >52 to odjąc od wyniku ilość tyg w danym roku - def 52
obciazenie_df2['Weeks_left'] = (obciazenie_df2['Finnish_week']) - (obciazenie_df2['Pre_fabrication_plan'])

In [36]:
#???????Czy liczyć nadwyżkę 20% jako zabezpieczenie przy przekroczeniach??????
obciazenie_df2['Work_h_left'] = (obciazenie_df2['Wszystkie.technol']) - (obciazenie_df2['Wszystkie.wykonane'])

#Jeżeli zlecenie zlecenie przekroczyło godziny praca liczona jest w nadgodzinach - przyjmuje się 0 do obciażenia

In [37]:
#Podzielenie pozostałej pracy równomiernie na tygodnie
obciazenie_df2['Workload_per_week'] = (obciazenie_df2['Work_h_left']) / (obciazenie_df2['Weeks_left'])
obciazenie_df2['Workload_per_week']

0    -0.169202
1    -0.388889
2    -0.780172
3     2.370370
4     9.242857
        ...   
58         NaN
59         NaN
60         NaN
61         NaN
62         NaN
Name: Workload_per_week, Length: 63, dtype: float64

In [38]:
#Podział obciążenia na tygodnie obecny+13 tyg
obciazenie_df3 = obciazenie_df2[["No", "Workload_per_week"]]

x = current_week
for i in range(0,13):
    obciazenie_df3[x+i]=0

obciazenie_df3

<ipython-input-38-746ae2e974f9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obciazenie_df3[x+i]=0


,No,Workload_per_week,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124
0,191,-0.169202,0,0,0,0,0,0,0,0,0,0,0,0,0
1,192,-0.388889,0,0,0,0,0,0,0,0,0,0,0,0,0
2,870,-0.780172,0,0,0,0,0,0,0,0,0,0,0,0,0
3,908,2.370370,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1015,9.242857,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1268,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
59,1269,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
60,1270,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
61,1271,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
#Dodanie obciazenia zarezerwowanego na dany tydzien w zależności od rozpoczecia i zakonczenia danego zlecenia
#If prefabrication plan is >column index & <= finnish week -> column index.No zlecenia = Workload_per_week

In [ ]:
#Zrobienie wykresu